<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/dataset_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create a script to load the audio dataset from drive

In [1]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## import dependencies
import os
import sys
import torch
import torch.nn as nn
import torchaudio
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## load locally stored musdb18 dataset using torchaudio
drive_path = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/audio/at_mic_music/train'
data_path = os.path.join(os.getcwd(), drive_path)
#atmicdataset = torchaudio.datasets.MUSDB18(data_path, download=True)

In [ ]:
data_path = os.path.join(os.getcwd(), drive_path)
print(data_path)
#atmicdataset = torchaudio.datasets.MUSDB18(data_path, download=True)

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024/audio/at_mic_music/train


Write code for dataset loader

Trying home cooked dataset loader

In [ ]:
from scipy.io import wavfile
class MusicDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.track_list = []
        for subdir, dirs, files in os.walk(root_dir):
          for dir in dirs:
              self.track_list.append(os.path.join(subdir, dir))

    def __len__(self):
        return len(self.track_list)

    def __getitem__(self, index):
        file_path = self.track_list[index]
        fs, bass  = wavfile.read(os.path.join(file_path, "bass.wav"))
        _, drums = wavfile.read(os.path.join(file_path, "drums.wav"))
        _, vocals = wavfile.read(os.path.join(file_path, "vocals.wav"))
        _, other = wavfile.read(os.path.join(file_path, "other.wav"))
        _, mix = wavfile.read(os.path.join(file_path, "mixture.wav"))

        bass = torch.from_numpy(bass).float()
        drums = torch.from_numpy(drums).float()
        vocals = torch.from_numpy(vocals).float()
        other = torch.from_numpy(other).float()
        mix = torch.from_numpy(mix).float()

        return bass, drums, vocals, other, mix, fs

# Load the music dataset into a PyTorch audio dataset
music_dataset = MusicDataset(root_dir=data_path)

# Create a PyTorch data loader
data_loader = torch.utils.data.DataLoader(music_dataset, batch_size=1, shuffle=True)

# Iterate over the data loader
#for batch in data_loader:
#    bass, drums, vocals, other, mix = batch

    # Process the audio data here

In [ ]:
bass, drums, vocals, other, mix, fs = music_dataset[0]

In [ ]:
print(fs)

44100


In [ ]:
from IPython.display import Audio
print(np.shape(mix))
Audio(other[:fs*10,0], rate=fs)

torch.Size([7560512, 2])


Create transforms for converting to spectrograms and cropping to specific sizes, torchaudio may have these transforms already! Find and use them.

Then you can use dataloader for the subsequent parts

In [ ]:
data_loader = torch.utils.data.DataLoader(music_dataset, batch_size=1, shuffle=True)
for data in data_loader:
  print(data[0].size())
  print('sample rate = ', data[5])
  break

torch.Size([1, 10901953, 2])
sample rate =  tensor([44100])
